In [1]:
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.utils import plot_model
model = ResNet50V2(include_top=False, input_shape=(224, 224, 3))
plot_model(model, to_file="res.png", show_shapes=True)
#畫出model,可以看出結構:有兩種res,一種是通道數有增加的圈,另一種則沒有

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# 通道數有增加的圈>>>用一個短路線來調整通道數
# 短路線: 1 x 1捲機(256)
w = model.get_layer("conv2_block1_0_conv").get_weights()
print(w[0].shape)
# 殘差線: 1 x 1捲機(64) -> 3 x 3捲機(64) -> 1 x 1捲機(256)
w = model.get_layer("conv2_block1_1_conv").get_weights()
print(w[0].shape)
w = model.get_layer("conv2_block1_2_conv").get_weights()
print(w[0].shape)
w = model.get_layer("conv2_block1_3_conv").get_weights()
print(w[0].shape)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation
from tensorflow.keras.layers import Input, Add

i = Input(shape=(56, 56, 64))
# 短路
x1 = Conv2D(256, 1, padding="same")(i)
# 殘差
#這裡的1,1卷積是在重新組合
x2 = Conv2D(64, 1, padding="same")(i)
x2 = BatchNormalization()(x2)
x2 = Activation("relu")(x2)
#做3,3卷積,最後想把通道數擴增成256,但如果在這裡一起做,計算量太大
x2 = Conv2D(64, 3, padding="same")(x2)
x2 = BatchNormalization()(x2)
x2 = Activation("relu")(x2)
#做回1,1卷積,並且把通道數擴成256,減少計算量
x2 = Conv2D(256, 1, padding="same")(x2)
#最後一層不能做Activation("relu"),必定是正值(0~1),如果input>output要變小,就會沒有辦法
# ++
o = Add()([x1, x2])
p = Model(inputs=i, outputs=o)
plot_model(p, show_shapes=True)

In [ ]:
# 沒有改變通道數的圈圈
# 殘差線: 1x1捲機(64) -> 3x3捲機(64) -> 1x1捲機(256)
w = model.get_layer("conv2_block2_1_conv").get_weights()
print(w[0].shape)
w = model.get_layer("conv2_block2_2_conv").get_weights()
print(w[0].shape)
w = model.get_layer("conv2_block2_3_conv").get_weights()
print(w[0].shape)

In [ ]:
i = Input(shape=(56, 56, 256))

# 殘差
x2 = Conv2D(64, 1, padding="same")(i)
x2 = BatchNormalization()(x2)
x2 = Activation("relu")(x2)
x2 = Conv2D(64, 3, padding="same")(x2)
x2 = BatchNormalization()(x2)
x2 = Activation("relu")(x2)
x2 = Conv2D(256, 1, padding="same")(x2)
# ++
o = Add()([i, x2])
p = Model(inputs=i, outputs=o)
plot_model(p, show_shapes=True)